# NLP wiht LSTM TensorFlow Youtube Tutorial 

Video: https://www.youtube.com/watch?v=ZMudJXhsUpY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=6

Code: https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%202%20-%20Notebook.ipynb#scrollTo=w9vH8Y59ajYL

In [1]:
traning_new_model = True                # Soll ein neues Modell traniert werden ?
copie_js_model_to_react = True          # Soll das js Modell direkt rüber kopiert werden ?

## Setup 

In [2]:
import tensorflow as tf
import keras 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import numpy as np 
import os

TF_USE_LEGACY_KERAS=1 # https://github.com/tensorflow/tfjs/issues/8328#issuecomment-2212490809

! echo $TF_USE_LEGACY_KERAS
! python3 --version
print(tf.__version__)
print(keras.__version__)


2024-11-06 17:39:25.824026: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 17:39:27.105890: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 17:39:27.106539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 17:39:27.224385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 17:39:27.530586: I tensorflow/core/platform/cpu_feature_guar

1
Python 3.10.15
2.15.1
2.15.0


In [3]:
# Modellname ohne führenden Schrägstrich
model_name = "lstm"  # ohne führendes "/"

text_file = os.path.abspath("../../example_text.txt")
model_file = os.path.join(os.path.abspath("./models"), model_name + ".h5")
js_model_dir = os.path.abspath("./models/lstm_js")
react_model_path = os.path.join(os.path.abspath("/workspace/silvan/webpage/silvan-webpage/public/models"), model_name + "_js")

print(react_model_path)

/workspace/silvan/webpage/silvan-webpage/public/models/lstm_js


## Data

In [4]:
# Datensatz laden
data = open(text_file, 'rb').read().decode(encoding='utf-8')

print(f"Inhalte des Datensatzes: \n\n{data[:250]}")

Inhalte des Datensatzes: 

Wie geht es dir?
Mir geht's gut, danke.
Was machst du heute?
Hast du gut geschlafen?
Wo wohnst du?
Ich wohne in Berlin.
Wie spät ist es?
Es ist schon ziemlich spät.
Hast du heute Zeit?
Was hast du am Wochenende vor?
Kannst du mir bitte helf


In [5]:
# Corpus erstellen
tokenizer = Tokenizer()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(f"Der Tokenizer kennt {total_words} Wörter")
print(f"Die Wörter werden folgenden Indexe zugenortnet: {tokenizer.word_index}")


Der Tokenizer kennt 200 Wörter
Die Wörter werden folgenden Indexe zugenortnet: {'\r': 1, 'du': 2, 'ich': 3, 'das': 4, 'hast': 5, 'habe': 6, 'was': 7, 'wie': 8, 'es': 9, 'ist': 10, 'heute': 11, 'schon': 12, 'kannst': 13, 'möchtest': 14, 'hier': 15, 'wann': 16, 'nicht': 17, 'lange': 18, 'kann': 19, 'wo': 20, 'in': 21, 'zeit': 22, 'zu': 23, 'noch': 24, 'zum': 25, 'der': 26, 'bin': 27, 'dir': 28, 'mir': 29, 'gut': 30, 'bitte': 31, 'etwas': 32, 'gehen': 33, 'muss': 34, 'war': 35, 'machst': 36, 'spät': 37, 'ziemlich': 38, 'wochenende': 39, 'frei': 40, 'wir': 41, 'uns': 42, 'mich': 43, 'schön': 44, 'den': 45, 'film': 46, 'viel': 47, 'tun': 48, 'mit': 49, 'keine': 50, 'einen': 51, 'dauert': 52, 'finde': 53, 'die': 54, 'nächste': 55, 'gibt': 56, 'komme': 57, 'bleibst': 58, 'gleich': 59, 'an': 60, 'meinen': 61, 'vergessen': 62, 'dein': 63, 'morgen': 64, 'bus': 65, 'weit': 66, 'bis': 67, 'eine': 68, 'geht': 69, "geht's": 70, 'danke': 71, 'geschlafen': 72, 'wohnst': 73, 'wohne': 74, 'berlin': 75, 

In [6]:
# Datensatu generieren
input_sequences = []

# Wandelt jede Zeile in eine Liste von Wortindizes 
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]

	# n-Gramm-Sequenz generieren
	for i in range(1, len(token_list)):

		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Alle Sequences auf gleiche Laenge bringen 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Aufteilen in Prädiktoren und Labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

## Modell

In [7]:
# Modell Struktur 

from tensorflow.keras.models import load_model

def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=total_words, output_dim=100))
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(total_words, activation='softmax'))
    
    # Optimizer und Modell-Compile
    adam = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# Prüfen, ob das Modell bereits existiert
if os.path.exists(model_file) and traning_new_model == False:
    print("Modell gefunden. Lade bestehendes Modell...")
    model = load_model(model_file)  
    
else:
    if traning_new_model ==True:
        print("Neuse Model soll traniert werden")
    else: 
        print("Altes Modell nicht gefunden")
    
    print("Startet Training \n\n")
    model = create_model()  # Neues Modell erstellen
    
    # Modell trainieren
    history = model.fit(xs, ys, epochs=100, verbose=1)
    
    # Modell speichern
    model.save(model_file)
    print(f"Modell gespeichert unter: {model_file}")



Neuse Model soll traniert werden
Startet Training 


Epoch 1/100
15/15 [==============================] - 5s 25ms/step - loss: 4.9905 - accuracy: 0.1809
Epoch 2/100
15/15 [==============================] - 0s 17ms/step - loss: 4.3596 - accuracy: 0.2383
Epoch 3/100
15/15 [==============================] - 0s 17ms/step - loss: 3.7683 - accuracy: 0.2979
Epoch 4/100
15/15 [==============================] - 0s 19ms/step - loss: 3.3252 - accuracy: 0.3085
Epoch 5/100
15/15 [==============================] - 0s 25ms/step - loss: 2.9526 - accuracy: 0.3617
Epoch 6/100
15/15 [==============================] - 0s 19ms/step - loss: 2.6766 - accuracy: 0.3723
Epoch 7/100
15/15 [==============================] - 0s 26ms/step - loss: 2.3455 - accuracy: 0.4106
Epoch 8/100
15/15 [==============================] - 1s 35ms/step - loss: 2.0632 - accuracy: 0.4404
Epoch 9/100
15/15 [==============================] - 0s 28ms/step - loss: 1.7340 - accuracy: 0.5298
Epoch 10/100
15/15 [===========================

/usr/local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Modell gespeichert unter: /workspace/algorithem/next_word_prediciton/rnns/models/lstm.h5


## Verwendung 

### Text generieren

In [8]:
seed_text = "Hallo ich bin"
next_words = 1
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 1s 1s/step
Hallo ich bin sehr


### Nextes Word vorhesagen  

In [9]:
import numpy as np

seed_text = "Wie"
top_k = 10  # Anzahl der Top-Wörter

# Tokenisierung und Padding des Eingabetexts
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

# Vorhersagen berechnen
predictions = model.predict(token_list, verbose=0)[0]  # Softmax-Ausgabe

# Top-10 wahrscheinlichste Wörter finden
top_indices = np.argsort(predictions)[-top_k:][::-1]  #Sortiere absteigend

# Wörter und Wahrscheinlichkeiten sammeln
top_words = [(word, predictions[index]) for word, index in tokenizer.word_index.items() if index in top_indices]

# Sortiere die Wörter nach Wahrscheinlichkeit in absteigender Reihenfolge
top_words = sorted(top_words, key=lambda x: x[1], reverse=True)

# Ausgabe der Top-10 Wörter und Wahrscheinlichkeiten
print(f"Top {top_k} nächste Wörter für '{seed_text}':")
for word, prob in top_words:
    print(f"{word}: {(prob * 100):.4f} %")


Top 10 nächste Wörter für 'Wie':
lange: 35.6560 %
war: 21.3266 %
komme: 10.4144 %
geht: 9.6432 %
groß: 7.9449 %
spät: 7.0866 %
weit: 6.8149 %
ist: 0.1222 %
hast: 0.0668 %
finde: 0.0339 %


## Konvertierung 

In [10]:
if traning_new_model:
    print("Konveriert neues Modell \n\n")
    ! tensorflowjs_converter --input_format=keras {model_file} {js_model_dir}
else:
    print("Konvertiertes Model bereits vorhanden")


Konveriert neues Modell 


/bin/bash: line 1: tensorflowjs_converter: command not found


In [11]:
if copie_js_model_to_react:
    print("Kopiere JS-Model in React-Model")
    ! cp -r {js_model_dir} {react_model_path}
else:
    print("JS-Model bereits im React-Model vorhanden")

Kopiere JS-Model in React-Model
cp: cannot create directory '/workspace/silvan/webpage/silvan-webpage/public/models/lstm_js': No such file or directory
